### + 버튼으로 위젯 새창 띄우기

위젯창을 새로 띄우는 "메모추가" 기능을 구현하기 위해서는 이해하고 넘어가야할 부분이 있습니다.
<pre style="background-color:#eeeeee">
app = QtWidgets.QApplication(sys.argv)
memo = MyMemo()
sys.exit(app.exec_())
</pre>
일반적인 PySide2 프로젝트는 위의 코드에서처럼 ```QApplication()``` 메인 인스턴스를 생성하고 ```memo = MyMemo()``` 우리가 작성한 클래스를 생성하여 ```app.exec_()``` 메인 인스턴스를 실행하는 과정으로 진행됩니다. ```QApplication()``` 은 쉽게 프로그램의 메인 프로세스 개념으로 봐야하고 이 메인프로세스 안에서 ```MyMemo()``` 객체가 운영되는 개념으로 이해할 수 있습니다.

그렇다면 "메모추가" 기능을 구현하기 위해선 위에서 우리가 ```MyBar```에 작성해놓은 ```+``` 버튼을 클릭했을때 위의 코드에서 ```memo=MyMemo()``` 부분이 따로 수행되어야 하며 이렇게 실행된 메모위젯의 ```memo``` 변수를 관리해야 합니다.

In [ ]:
class MyApp():
    def __init__(self):
        self.app = QtWidgets.QApplication(sys.argv)
        self.windows = []
        self.initMemo()

    def initMemo(self):
        self.create_new_memo()

    def create_new_memo(self):
        w = MyMemo(self.create_new_memo)
        w.show()
        self.windows.append(w)

- ```MyApp()``` 클래스의 멤버 변수 ```self.app=QtWidgets.QApplication()``` 으로 메인 프로세스를 클래스 내부에서 생성합니다.
- 이렇게 생성된 메모 위젯은 ```self.windows``` 라는 리스트형 변수에 의해 관리 됩니다.
- ```def create_new_memo(self)``` 함수내에서 ```MyMemo``` 객체를 생성하는데 이때 생성자에 ```create_new_memo``` 함수를 인자로 넘겨줍니다.

<pre style="background-color:#eeeeee">
class MyMemo(QtWidgets.QWidget):
    def __init__(self, <b>on_create</b>):
        super().__init__()
        <b>self.on_create = on_create</b>
        self.color_text = "#000000"
        self.color_bg = "#dce459"
        self.initUI()
        
class MyBar(QtWidgets.QWidget):
    def btn_new_clicked(self):
        <b>self.parent.on_create()</b>
</pre>
- ```MyApp``` 에서 ```MyMemo``` 생성시에 넘긴 인자값은 위의 코드에서처럼 ```MyMemo``` 생성자에서 받아서 처리 합니다.

### 데이터베이스 생성

메모를 작성하고 종료할때 그리고 프로그램이 다시 실행될때 메모 내용을 기억하고 있으려면 어딘가에 저장을 해야 합니다. 일반적인 텍스트 파일을 사용해서 저장을 할 수도 있겠지만 이전 강좌에서 배운 SQLite 를 활용해서 메모 저장 기능을 만들어보도록 하겠습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
class MyApp():
    def __init__(self):
        ...코드 생략....
        self.cur_dir = os.path.dirname(__file__)
        self.db_name = self.cur_dir + "\\" + "memo.db"

    def initDB(self):
        self.con = sql.connect(self.db_name)
        self.cursor = self.con.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS memo (
            '_idx' INTEGER PRIMARY KEY AUTOINCREMENT,
            '_title' VARCHAR(500),
            '_memo' TEXT,
            '_pubdate' TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            '_pos_x' INTEGER,
            '_pos_y' INTEGER,
            '_width' INTEGER,
            '_height' INTEGER,
            '_color_bg' VARCHAR(30),
            '_color_text' VARCHAR(30)
        )"""
        self.con.execute(query)
        self.con.close()
</pre>

- 데이터베이스 생성 ```initDB()``` 는 메모 위젯과 상관없이 프로그램이 실행되면 생성되어야 하는 부분입니다.
- 데이터베이스 테이블에는 ```메모인덱스, 타이틀, 메모, 생성일자, 메모창 x위치, 메모창 y위치, 메모창의 폭, 메모창의 높이, 메모창의 배경색, 메모창의 글자색``` 테이블을 구현했습니다. 여기서 타이틀, 글자색 같은건 당장 사용되지는 않습니다만 추후 확장성을 고려해서 만들어놓기만 했습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
def loadDB(self):
    query = 'SELECT _idx, _title, _memo, _pos_x, _pos_y, _width, _height, _color_bg, _color_text FROM memo'

    with sql.connect(self.db_name) as con:
        cur = con.cursor()
        cur.execute(query)
        memos = cur.fetchall()
        con.commit()
        cur.close()

    for m in memos:
        idx, title, memo, x, y, w, h, c_bg, c_text = m
        self.create_new_memo(idx, memo, title, QtCore.QRect(x, y, w, h), c_bg, c_text)
    return len(memos)
</pre>
        
- ```loadDB()``` 함수에는 메모프로그램 실행시에 SQL ```SELECT``` 문을 이용하여 현재 데이터베이스의 전체 메모 데이터를 구해와서 해당 메모 마다의 메모위젯을 생성하는 내용입니다. 기존의 작성된 ```create_new_memo()``` 함수에 필요한 값을 모두 전달해야 합니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
def create_new_memo(self, idx=None, text=None, title=None, rect=None, color_bg=None, color_text=None):
    w = MyMemo(self.create_new_memo, 
                self.close_memo, self.delete_memo, 
                idx, 
                text, 
                title, 
                rect, 
                color_bg=color_bg, 
                color_text=color_text)
    w.show()
    self.windows.append(w)
</pre>

- 여기서 create_new_memo 함수로 넘어온 인자값들을 ```MyMemo``` 에서도 처리할 수 있게끔 코드를 수정해야 합니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
class MyMemo(QtWidgets.QWidget):
    def __init__(self, on_create, idx=None, text=None, title=None, rect=None, color_bg=None, color_text=None):
        super().__init__()
        self.on_create = on_create   # 메모 추가 버튼 클릭시 호출될 함수
        self.text = text
        self.title = title
        self.rect = rect
        self.deleted = False
        self.idx = idx

        self.color_text = "#000000" if color_text is None else color_text
        self.color_bg = "#dce459" if color_bg is None else color_bg
</pre>

- 메모 위젯이 생성될때 생성자로 넘어온 ```idx, text``` 등의 값으로 해당 위젯을 생성합니다.